## Find matching coordinates  - load source zips
Haetaan koalatietojen tiedot ja hilatiedot sekä selvitetään niiden perusteella yhteiset koordinaatit.
#Load the actual sample area data (koeala) and grid data (hila) and find which 16m x 16 m grid is surrounding the sample coordinates.
#Load source zip files direcly from web site (not use local computer upload)


In [7]:
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git


  Cloning git://github.com/geopandas/geopandas.git to /tmp/pip-req-build-iwzx9shk
  Running command git clone -q git://github.com/geopandas/geopandas.git /tmp/pip-req-build-iwzx9shk
     |████████████████████████████████| 14.8MB 318kB/s 
     |████████████████████████████████| 6.5MB 43.6MB/s 
  Created wheel for geopandas: filename=geopandas-0.9.0+7.gf5c54ed-py2.py3-none-any.whl size=995043 sha256=c1813b12f623140a655f6959a7a14b204f047ce37905c425d28be2ff710b8d81
  Stored in directory: /tmp/pip-ephem-wheel-cache-zbfmhsu8/wheels/91/24/71/376c9c67192694168352afcccc2d264248f7e2cc6192997186
Successfully built geopandas


In [8]:
# Libraries (not all may be necessary)
import geopandas #needed to open gpkg source date file 
#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import fiona
#import seaborn as sns

In [9]:
!wget https://aineistot.metsaan.fi/avoinmetsatieto/Kaukokartoituskoealat/Kunta/Koeala_Sipoo.zip


--2021-03-22 09:18:29--  https://aineistot.metsaan.fi/avoinmetsatieto/Kaukokartoituskoealat/Kunta/Koeala_Sipoo.zip
Resolving aineistot.metsaan.fi (aineistot.metsaan.fi)... 131.207.79.68
Connecting to aineistot.metsaan.fi (aineistot.metsaan.fi)|131.207.79.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 165249 (161K) [application/x-zip-compressed]
Saving to: ‘Koeala_Sipoo.zip’

Koeala_Sipoo.zip    100%[===================>] 161.38K   151KB/s    in 1.1s    

2021-03-22 09:18:31 (151 KB/s) - ‘Koeala_Sipoo.zip’ saved [165249/165249]



In [10]:
!unzip Koeala_Sipoo.zip

Archive:  Koeala_Sipoo.zip
  inflating: Koeala_Sipoo.gpkg       


In [11]:
!ls

drive  Koeala_Sipoo.gpkg  Koeala_Sipoo.zip  new.csv  sample_data


In [12]:
#Load sample area data (koeala)
gdf_koe = geopandas.read_file('Koeala_Sipoo.gpkg', layer='sampleplot')
print ("Koeala coordinate reference system is:", gdf_koe.crs)
gdf_koe=gdf_koe["geometry"] # only keep relevant column in this analysis
print(gdf_koe)   

Koeala coordinate reference system is: epsg:3067
0      POINT (409803.260 6683110.170)
1      POINT (409612.250 6683121.040)
2      POINT (409968.980 6683185.040)
3      POINT (409870.720 6683241.770)
4      POINT (410005.090 6683296.690)
                    ...              
236    POINT (401609.480 6698446.710)
237    POINT (401990.100 6698575.810)
238    POINT (401624.280 6698600.960)
239    POINT (401791.930 6698604.860)
240    POINT (400036.306 6705775.531)
Name: geometry, Length: 241, dtype: geometry


/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [13]:
!wget https://aineistot.metsaan.fi/avoinmetsatieto/Hila/Kunta/Hila_Sipoo.zip

--2021-03-22 09:18:31--  https://aineistot.metsaan.fi/avoinmetsatieto/Hila/Kunta/Hila_Sipoo.zip
Resolving aineistot.metsaan.fi (aineistot.metsaan.fi)... 131.207.79.68
Connecting to aineistot.metsaan.fi (aineistot.metsaan.fi)|131.207.79.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 101176619 (96M) [application/x-zip-compressed]
Saving to: ‘Hila_Sipoo.zip’

Hila_Sipoo.zip      100%[===================>]  96.49M   352KB/s    in 4m 44s  

2021-03-22 09:23:16 (348 KB/s) - ‘Hila_Sipoo.zip’ saved [101176619/101176619]



In [14]:
!unzip Hila_Sipoo.zip
!ls

Archive:  Hila_Sipoo.zip
  inflating: Hila_Sipoo.gpkg         
drive		 Hila_Sipoo.zip     Koeala_Sipoo.zip  sample_data
Hila_Sipoo.gpkg  Koeala_Sipoo.gpkg  new.csv


In [15]:
!ogr2ogr -f "CSV" Hila_Sip.csv Hila_Sipoo.gpkg -lco GEOMETRY=AS_WKT

In [16]:
!ls

drive	      Hila_Sipoo.gpkg  Koeala_Sipoo.gpkg  new.csv
Hila_Sip.csv  Hila_Sipoo.zip   Koeala_Sipoo.zip   sample_data


In [17]:
#Load grid data (hila)

#Run command line conversion from gpkg to csv with
#ogr2ogr -f "CSV" Hila_Mie.csv Hila_Mie.gpkg -lco GEOMETRY=AS_WKT
#This is only needed when to be done once when the data is loaded or updated

from shapely import wkt
import io

path = "./Hila_Sip.csv"  # Define the file path
gdf_hila = pd.read_csv(path)
print(gdf_hila)
gdf_hila['geometry']= gdf_hila['WKT'].apply(wkt.loads)
gdf_hila = geopandas.GeoDataFrame(gdf_hila, geometry='geometry') 
gdf_hila = gdf_hila["geometry"] #keep only relevant column in analysis
gdf_hila.crs = "EPSG:3067"
print ("Hila coordinate reference system is:", gdf_hila.crs)
print(gdf_hila)

                                                      WKT  ...              updatetime
0       POLYGON ((404240.0 6683984.0,404256.0 6683984....  ...  2020/09/28 20:03:00+00
1       POLYGON ((404256.0 6683984.0,404272.0 6683984....  ...  2020/09/28 20:03:00+00
2       POLYGON ((404384.0 6683984.0,404400.0 6683984....  ...  2020/09/25 11:42:45+00
3       POLYGON ((404400.0 6683984.0,404416.0 6683984....  ...  2020/09/25 11:42:45+00
4       POLYGON ((404416.0 6683984.0,404432.0 6683984....  ...  2020/09/25 11:42:45+00
...                                                   ...  ...                     ...
670633  POLYGON ((400864.0 6677728.00000014,400880.0 6...  ...  2016/09/05 07:44:10+00
670634  POLYGON ((400848.0 6677728.00000014,400864.0 6...  ...  2016/09/05 07:44:10+00
670635  POLYGON ((400816.0 6677728.00000014,400832.0 6...  ...  2016/09/05 07:44:10+00
670636  POLYGON ((400880.0 6677728.00000014,400896.0 6...  ...  2016/09/05 07:44:10+00
670637  POLYGON ((400832.0 6677728.00000014

In [ ]:
#Loop over all sample area data items and in another loop to find if any of grid data areas match
#This code takes couple of days to run in laptop. Better solutions would be nice to have
a=[]
for i in gdf_koe.index:
    xs=gdf_koe.geometry.iloc[i].x
    ys=gdf_koe.geometry.iloc[i].y
    for n in range(len(gdf_hila)):
        xn=np.squeeze(np.dstack((gdf_hila.geometry.iloc[n].exterior.xy)))[0,0]
        xn1=np.squeeze(np.dstack((gdf_hila.geometry.iloc[n].exterior.xy)))[2,0]
        yn=np.squeeze(np.dstack((gdf_hila.geometry.iloc[n].exterior.xy)))[0,1]
        yn1=np.squeeze(np.dstack((gdf_hila.geometry.iloc[n].exterior.xy)))[2,1]
        if ((xn <= xs) and (xn1 > xs) and (yn <= ys) and (yn1 > ys)):
            print(i)
            print(n)
            a.append(i)
            a.append(n)


6
345765
29
346429
30
346443
102
598367
103
523005
104
531770
105
531194
106
519950
107
525499
108
521698
114
134600


In [ ]:
import csv

#Store output as a file in Colab
with open('sip_match.csv', 'w') as f:
    for d in a:
        f.write(str(d))
        f.write("\n")

#Store output as a file in Google Drive
from google.colab import drive
drive.mount('/content/drive')

with open('/content/drive/My Drive/frd/sip_match.csv', 'w') as f:
  for d in a:
    f.write(str(d))
    f.write("\n")


In [ ]:
!ls

# New Section